Let's solve a rubik's cube.

In [391]:
import numpy as np
import torch
import matplotlib.pyplot as plt

class Cube:
    def __init__(self):
        self.cube = np.zeros((5,5,5))
        self.cube[0,1:4,1:4] += 2
        self.cube[-1,1:4,1:4] += 1
        self.cube[1:4,1:4,0] += 3
        self.cube[1:4,1:4,-1] += 4
        self.cube[1:4,0,1:4] += 5
        self.cube[1:4,-1,1:4] += 6

        self.id = 0

        self.moves = {
            'n':self.n,
            'f':self.f,
            'F':self.F,
            's':self.s,
            'S':self.S,
            'b':self.b,
            'B':self.B,
            'l':self.l,
            'L':self.L,
            'm':self.m,
            'M':self.M,
            'r':self.r,
            'R':self.R,
            'u':self.u,
            'U':self.U,
            'e':self.e,
            'E':self.E,
            'd':self.d,
            'D':self.D,
        }

    @property
    def n(self):
        # null
        return self

    @property
    def f(self):
        # front clockwise
        self.cube[-2] = np.flip(self.cube[-2].T,1)
        self.cube[-1] = np.flip(self.cube[-1].T,1)
        return self

    @property
    def F(self):
        # front counter-clockwise
        self.cube[-2] = np.flip(self.cube[-2].T,0)
        self.cube[-1] = np.flip(self.cube[-1].T,0)
        return self

    @property
    def s(self):
        # mid x-y clockwise
        self.cube[2] = np.flip(self.cube[2].T,0)
        return self

    @property
    def S(self):
        # mid x-y counter-clockwise
        self.cube[2] = np.flip(self.cube[2].T,1)
        return self

    @property
    def b(self):
        # back clockwise
        self.cube[1] = np.flip(self.cube[1].T,0)
        self.cube[0] = np.flip(self.cube[0].T,0)
        return self

    @property
    def B(self):
        # back counter-clockwise
        self.cube[1] = np.flip(self.cube[1].T,1)
        self.cube[0] = np.flip(self.cube[0].T,1)
        return self

    @property
    def l(self):
        # left clockwise
        self.cube[:,:,:2] = np.rot90(self.cube[:,:,:2],axes=(0,1))
        return self

    @property
    def L(self):
        # left counter-clockwise
        self.cube[:,:,:2] = np.rot90(self.cube[:,:,:2],axes=(1,0))
        return self

    @property
    def m(self):
        # mix y-z clockwise
        self.cube[:,:,2] = np.rot90(self.cube[:,:,2],axes=(0,1))
        return self

    @property
    def M(self):
        # mix y-z counter-clockwise
        self.cube[:,:,2] = np.rot90(self.cube[:,:,2],axes=(1,0))
        return self

    @property
    def r(self):
        # right clockwise
        self.cube[:,:,3:] = np.rot90(self.cube[:,:,3:],axes=(0,1))
        return self

    @property
    def R(self):
        # right counter-clockwise
        self.cube[:,:,3:] = np.rot90(self.cube[:,:,3:],axes=(1,0))
        return self

    @property
    def u(self):
        # up clockwise
        self.cube[:,1] = np.flip(self.cube[:,1].T,-1)
        self.cube[:,0] = np.flip(self.cube[:,0].T,-1)
        return self

    @property
    def U(self):
        # up counter-clockwise
        self.cube[:,1] = np.flip(self.cube[:,1].T,0)
        self.cube[:,0] = np.flip(self.cube[:,0].T,0)
        return self

    @property
    def e(self):
        # mid x-z clockwise
        self.cube[:,2] = np.flip(self.cube[:,2].T,-1)
        return self

    @property
    def E(self):
        # mid x-z counter-clockwise
        self.cube[:,2] = np.flip(self.cube[:,2].T,0)
        return self

    @property
    def d(self):
        # down clockwise
        self.cube[:,-2] = np.flip(self.cube[:,-2].T,0)
        self.cube[:,-1] = np.flip(self.cube[:,-1].T,0)
        return self

    @property
    def D(self):
        # down counter-clockwise
        self.cube[:,-2] = np.flip(self.cube[:,-2].T,-1)
        self.cube[:,-1] = np.flip(self.cube[:,-1].T,-1)
        return self

emb_size = 6
C = Cube()
corners = dict()
cornerRoots = [C.cube[0+c*3:2+c*3,0+b*3:2+b*3,0+a*3:2+a*3] for a in range(2) for b in range(2) for c in range(2)]

for index,n in enumerate(cornerRoots):
    corners[np.sum(n**2)] = (
        index, # index
        dict(((k,j) for j,k in enumerate(int(x) for x in n.reshape((8)) if x))), #
    )

cornerPositions = np.array([np.sum(C.cube[0+c*3:2+c*3,0+b*3:2+b*3,0+a*3:2+a*3]**2) for a in range(2) for b in range(2) for c in range(2)]).astype(int)
cornersLabel = [corners[i] for i in cornerPositions]
bagOfCubes = {0:[C,torch.randn((emb_size,))]}

In [392]:
cornerPositions = np.array([np.sum(C.cube[0+c*3:2+c*3,0+b*3:2+b*3,0+a*3:2+a*3]**2) for a in range(2) for b in range(2) for c in range(2)]).astype(int)
cornersLabel = [corners[i] for i in cornerPositions]
cornersLabel

[(0, {2: 0, 5: 1, 3: 2}),
 (1, {5: 0, 3: 1, 1: 2}),
 (2, {2: 0, 3: 1, 6: 2}),
 (3, {3: 0, 6: 1, 1: 2}),
 (4, {2: 0, 5: 1, 4: 2}),
 (5, {5: 0, 4: 1, 1: 2}),
 (6, {2: 0, 4: 1, 6: 2}),
 (7, {4: 0, 6: 1, 1: 2})]

In [393]:
digit = {2:'0',3:'1',5:'2'}

In [394]:
C.R.D.r.d
cornerPositions = np.array([np.sum(C.cube[0+c*3:2+c*3,0+b*3:2+b*3,0+a*3:2+a*3]**2) for a in range(2) for b in range(2) for c in range(2)]).astype(int)
cornersLabel = [corners[i] for i in cornerPositions]
cornersLabel

[(0, {2: 0, 5: 1, 3: 2}),
 (1, {5: 0, 3: 1, 1: 2}),
 (4, {2: 0, 5: 1, 4: 2}),
 (3, {3: 0, 6: 1, 1: 2}),
 (2, {2: 0, 3: 1, 6: 2}),
 (5, {5: 0, 4: 1, 1: 2}),
 (7, {4: 0, 6: 1, 1: 2}),
 (6, {2: 0, 4: 1, 6: 2})]

In [395]:
corners

{38.0: (0, {2: 0, 5: 1, 3: 2}),
 35.0: (1, {5: 0, 3: 1, 1: 2}),
 49.0: (2, {2: 0, 3: 1, 6: 2}),
 46.0: (3, {3: 0, 6: 1, 1: 2}),
 45.0: (4, {2: 0, 5: 1, 4: 2}),
 42.0: (5, {5: 0, 4: 1, 1: 2}),
 56.0: (6, {2: 0, 4: 1, 6: 2}),
 53.0: (7, {4: 0, 6: 1, 1: 2})}